In [1]:
#extracts the raw ROI & outputs z-scored and smooth ROI activity 

In [1]:
from wfield import *
import os 
import h5py
import pandas as pd
from scipy.io import loadmat
from scipy.signal import savgol_filter

In [2]:
directory = "C:/Behavior/Animals/"
animal = "FV-4/"
maze = "SD/"
session = "12-Jul-2022/"
file = "{}"

In [3]:
def extract_raw_ROI(img_session):
    path = os.path.join(directory, animal, maze, session, file.format(img_session))
    os.chdir(path)
    print(path)
    U = np.load('U.npy')
    SVT = np.load('SVTcorr.npy')
    stack = SVDStack(U,SVT)
    lmarks = load_allen_landmarks('dorsal_cortex_landmarks.json')

    ccf_regions_reference,proj,brain_outline = allen_load_reference('dorsal_cortex')
    # the reference is in allen CCF space and needs to be converted
    # this converts to warped image space (accounting for the transformation)
    ccf_regions = allen_transform_regions(None,ccf_regions_reference,
                                          resolution = lmarks['resolution'],
                                            bregma_offset = lmarks['bregma_offset'])
    atlas, areanames, brain_mask = atlas_from_landmarks_file('dorsal_cortex_landmarks.json') # this loads the atlas in transformed coords

    # this does the transform (warps the original images)
    #stack.set_warped(1, M = lmarks['transform']) # this warps the spatial components in the stack
    
    # this converts the reference to image space (unwarped)
    atlas_im, areanames, brain_mask = atlas_from_landmarks_file('dorsal_cortex_landmarks.json',do_transform = True) # this loads the untransformed atlas
    ccf_regions_im = allen_transform_regions(lmarks['transform'],ccf_regions_reference,
                                            resolution = lmarks['resolution'],
                                            bregma_offset = lmarks['bregma_offset'])
    
    stack.set_warped(True) # once this is done once the transform is set and you can alternate between the 2 modes.
    activityR = []
    activityL = []
    for area in range(33):
        right = stack.get_timecourse(np.where(atlas == area)).mean(axis = 0)
        left = stack.get_timecourse(np.where(atlas == -area)).mean(axis = 0)
        activityR.append(right)
        activityL.append(left)
    
    #save activity matrices for easier access 

    np.save('activityR', activityR)
    np.save('activityL', activityL)

In [4]:
def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [5]:
def z_score_smooth(img_session):
    file = "{}"
    path = os.path.join(directory, animal, maze, session, file.format(img_session))
    print(path)
    os.chdir(path)
    #load activity matrices
    activityR = np.load('activityR.npy')
    activityL = np.load('activityL.npy')
    
    activityR_zscored = []
    activityR = np.load('activityR.npy')

    for area in range(33):
        activityR_series = pd.Series(activityR[area])
        x = zscore(activityR_series, 150)
        activityR_zscored.append(x.to_numpy())
    
    activityL_zscored = []
    activityL = np.load('activityL.npy')

    for area in range(33):
        activityL_series = pd.Series(activityL[area])
        x = zscore(activityL_series, 150)
        activityL_zscored.append(x.to_numpy())
    
    activityR_zscore_smooth = savgol_filter(activityR_zscored, 6, 1) #Applies a Savitzky-Golay filter to the z-scored array.
    activityL_zscore_smooth = savgol_filter(activityL_zscored, 6, 1)
    
    #save activity matrices for easier access 
    np.save('activityR_zscored', activityR_zscored)
    np.save('activityL_zscored', activityL_zscored)

In [6]:
extract_raw_ROI(1)
z_score_smooth(1)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/1
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/1


In [7]:
extract_raw_ROI(2)
z_score_smooth(2)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/2
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/2


In [8]:
extract_raw_ROI(3)
z_score_smooth(3)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/3
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/3


In [9]:
extract_raw_ROI(4)
z_score_smooth(4)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/4
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/4


In [10]:
extract_raw_ROI(5)
z_score_smooth(5)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/5
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/5


In [11]:
extract_raw_ROI(6)
z_score_smooth(6)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/6
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/6


In [12]:
extract_raw_ROI(7)
z_score_smooth(7)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/7
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/7


In [13]:
extract_raw_ROI(8)
z_score_smooth(8)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/8
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/8


In [14]:
extract_raw_ROI(9)
z_score_smooth(9)

C:/Behavior/Animals/FV-4/SD/12-Jul-2022/9
C:/Behavior/Animals/FV-4/SD/12-Jul-2022/9
